<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #10: Recommender Systems

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'"> andrey.saitgalin@gmail.com </span>

<h1 align="center">Organization Info</h1> 

* Дедлайн **18 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 494_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_401_task10.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``


--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Contest (100%)</h1> 
* Ссылка на контест: https://www.kaggle.com/t/f37413eb448145acba822f3a8c43946d

In [2]:
import numpy as np

import pandas as pd
from IPython.display import display

from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter


import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


%matplotlib inline

In [3]:
train = pd.read_csv("train.csv")
train.head()

,rating,movieId,userId
0,3.0,2158,10953099004185006703
1,3.5,12959,10953099004185006703
2,3.5,7808,10953099004185006703
3,4.0,9049,4762492897681260571
4,3.0,12363,2110017552365950661


In [4]:
train.userId = train.userId.astype('uint64')

In [5]:
train.head(2)

,rating,movieId,userId
0,3.0,2158,10953099004185006703
1,3.5,12959,10953099004185006703


In [6]:
display(train.size)
display(train.movieId.max())
display(len(set(train.userId)))

57043719

18345

138493

In [7]:
test = pd.read_csv("test.csv")
#test.userId = test.userId.astype('uint64')
test.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,movieId,userId
0,1,939,3128775.0
1,2,19,60588779.0
2,3,1213,278728.0
3,4,1141,141994.0
4,5,2935,595805.0


In [ ]:
user_to_compress_id = {}
compress_to_user_id = {}

def add_id_to_compress(i):
    if i not in user_to_compress_id:
        cid = len(compress_to_user_id)
        user_to_compress_id[i] = cid
        compress_to_user_id[cid] = i

for i, row in test.iterrows():
    add_id_to_compress(row.userId)
    
for i, row in train.iterrows():
    add_id_to_compress(row.userId)

In [ ]:
test["userId2"] = test["userId"]

ui2 = []

for i, row in test.iterrows():
    ui2.append(user_to_compress_id[row.userId])

test["userId2"] = np.array(ui2)

display(test.head())

In [ ]:
train["userId2"] = train["userId"]

ui2 = []

for i, row in train.iterrows():
    ui2.append(user_to_compress_id[row.userId])

train["userId2"] = np.array(ui2)

display(train.head())

In [ ]:
import time
from scipy.sparse.linalg import svds

df = train

# df["random"] = np.random.random(size=(df.shape[0], 1))
# idx = df.sort_values(by="random") \
#     .reset_index() \
#     .groupby("userId") \
#     .head(n=7)["index"]

# mask = df.index.isin(idx)
# train_df = df[~mask]
# test_df = df[mask]

train_df = df
    
train_M = csr_matrix(
    (train_df["rating"], (train_df["userId2"], train_df["movieId"] - 1)), shape=(138493, 26744))

In [ ]:
train_M_mean = train_M.mean()

#M2 = (train_M / train_M.mean(axis=1))

#U, S, Vt = svds(M2, k=100)
U, S, Vt = svds(train_M, k=100)

In [ ]:
class ErrorAggregator(object):
    def __init__(self):
        self.sum_ = 0.0
        self.count = 0
        
    def add_value(self, x, count):
        self.sum_ += x
        self.count += count
        
    def get_rmse(self):
        return np.sqrt(self.sum_ / (self.count - 1))
    
svd_error = ErrorAggregator()
mean_error = ErrorAggregator()
times = []

In [ ]:
for user_id, group in train[:10000].groupby("userId2"):
    start_time = time.time()
    
    uvec = U[user_id, :] * S
    
    movie_ids = group["movieId"] - 1
    ratings = group["rating"]
    
    diff = (uvec.dot(Vt[:, movie_ids]) - ratings.values) ** 2
    svd_error.add_value(np.sum(diff), len(diff))
    
    mean_diff = (ratings.values - train_M_mean) ** 2
    mean_error.add_value(np.sum(mean_diff), len(mean_diff))
    
    times.append(time.time() - start_time)

print("RMSE:", svd_error.get_rmse())
print("Random RMSE:", mean_error.get_rmse())
print("Average evaluation time:", np.mean(times))

In [ ]:
results = []

tt = train[:10000].copy()

for user_id, group in tt.groupby("userId2"):
    start_time = time.time()
    uvec = U[user_id, :] * S
    movie_ids = group["movieId"] - 1    
    results.append((user_id, movie_ids, uvec.dot(Vt[:, movie_ids])))

    
res2 = {}
for r in results:
    for m, ri in zip(r[1], r[2]):
        res2[(int(r[0]), int(m))] = ri
        
prating = []

for i, row in tt.iterrows():
    prating.append(res2[(int(row.userId2), int(row.movieId) - 1)])
    


prating = reshape(np.array(prating), hist_shape, hist_xs)
    
tt["rating2"] = np.array(prating)

print(tt.rating.mean())

plt.hist(tt.rating)
plt.show()

tt.head()

print((((tt["rating"] - tt["rating2"]) ** 2).sum() / len(tt)) ** 0.5)

In [ ]:
results = []

for user_id, group in test.groupby("userId2"):
    start_time = time.time()
    uvec = U[user_id, :] * S
    movie_ids = group["movieId"] - 1    
    results.append((user_id, movie_ids, uvec.dot(Vt[:, movie_ids])))

In [ ]:
res2 = {}
for r in results:
    for m, ri in zip(r[1], r[2]):
        res2[(int(r[0]), int(m))] = ri

In [ ]:
prating = []

for i, row in test.iterrows():
    prating.append(res2[(int(row.userId2), int(row.movieId) - 1)])

In [ ]:
prating = np.array(prating)
test["rating"] = prating

In [ ]:
answer = pd.read_csv('baseline.csv')
print(answer.rating.mean())
answer["rating"] = test["rating"]
print(answer.rating.mean())
answer.to_csv("to_send.csv", index=False)

In [ ]:
train.rating.mean()

In [ ]:
plt.hist(answer.rating, bins=50)
plt.show()

In [ ]:
answer.rating.mean()

In [ ]:
import numpy as np

In [ ]:

a = plt.hist(train.rating, bins=10)
plt.show()